In [1]:
import numpy as np
import pickle
from tqdm import tqdm
import re
import collections
import glob
from transformers import DistilBertTokenizer, DistilBertModel
import sys
import time
import requests
from IPython.display import display, HTML
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

sys.path.insert(0, '../src/models/')
sys.path.insert(0, '../src/features/')

from predict_model import loadBERT
from predict_model import SpanPredictor as classify
from build_features import text_cleaner, DuckDuckGo_Java, Bing_HTML, colorize_prediction

%matplotlib inline

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Load models

In [2]:
model = loadBERT("../models/", 'saved_weights_inf_FIXED_boot_beta80.pt')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

CPU Success


## Load the species list

In [3]:
plants_dict = collections.defaultdict(list)
root = '../data/processed/'
# Load the pickle list
data_files = glob.glob(root+ 'description*PLANTS.pkl')
for data_file in data_files:
    dict_ = pickle.load(open(data_file, 'rb'))
    for key, value in dict_.items():
        plants_dict[key] += value
    

# Order the dictionary based on the list length
plants_dict = collections.OrderedDict(sorted(plants_dict.items(), key= lambda x: len(x[1]), reverse=True))
# Correct first key
plants_dict['Poa'] = plants_dict.pop('oa')
# Get keys
plants = [key for key in plants_dict.keys()]

In [ ]:
len(plants_dict.keys())

## Iterate Species

In [ ]:
# Init dict
data = collections.defaultdict(list)
data.update(plants_dict)
# DEBUGGING
data_links = collections.defaultdict(list)
# Init driver
driver = webdriver.Safari()

queries = ['description', 'diagnosis', '', 'attributes', 'captions']

for species in tqdm(plants[500:1000]):
    # Empty list
    search_links = []
    
    for query in queries:
        # create query
        species_q = species.replace(' ', '+')
        species_q = f'"{species_q}"+{query}'
        try:
            search_links += DuckDuckGo_Java(species_q, 
                                            driver=driver)
            search_links += Bing_HTML(species_q)
         # Skip connection timeout
        except:
            continue
    # Drop duplicates
    search_links = list(set(search_links))
    # DEBUGGING
    data_links[species] += search_links
    # Loop over the URLs
    for URL in search_links:
        # Skip google archives
        if 'google' in URL:
            continue
        # PDF and TXT
        if URL.endswith('txt') or URL.endswith('pdf'):
            continue
        # Skip Plants of the world (already done)
        if 'powo' in URL:
            continue
        try:
            page = requests.get(URL, timeout=5)
            # Skip PDF files for now
            if page.headers['Content-Type'].startswith('application/pdf'):
                continue
            # Soup the result
            soup = BeautifulSoup(page.content, 'html.parser', from_encoding="iso-8859-1")    
            # Skip Embedded PDF's
            if 'pdf' in soup.title.text.lower():
                continue
            # Check if species exists somewhere within title
            if bool(set(species.split()).intersection(soup.title.text.split())):
                # Get text
                #dirty_text = soup.get_text(". ", strip=True)
                dirty_text = soup.get_text(" ", strip=False).replace('\n', '.')
                # Clean and break into sents
                sentences = text_cleaner(dirty_text)
                # Loop over the individual sentences
                for sentence in sentences:                    
                    # Create string object
                    sentence_str = str(sentence)
                    # Check if description
                    if classify(sentence_str, model=model):
                        data[species].append((sentence_str, URL))
                        # Drop duplicates
                        data[species] = list(set(data[species]))
        except: 
            continue
            
driver.quit()

  0%|                                                   | 0/500 [00:00<?, ?it/s]

In [6]:
# Dump pickle into file
with open('../data/description/03_descriptions_500-1000.pkl', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Change this to the dissimilar in future
counts = [len(sents) for key, sents in data.items()]

In [ ]:
len(counts)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.set_style('whitegrid')
ax = sns.histplot(counts, bins=50)
ax.set_title('Histogram')
#plt.xlim(0, 300)

In [ ]:
html = colorize_prediction(
    ['Something about a bird or plant',
     'The bill and the legs are both black.',
     'The house is large with enormous windows.',
     'This is something random, but the sexes are similar.',
     'Nuclear power might solve the energy power.',
     'The tree has a brown bark and the leaves are pointed',
     'By growing in shaded places, the plant reduces evaporation.',
     '3 cm seeds.'],
model=model)

In [ ]:
display(HTML(html))

In [ ]:
Html_file= open("test","w")
Html_file.write(xxx)
Html_file.close()

In [ ]:
xxx= html.replace('\n', '')

In [ ]:
URL = "https://www.beautyofbirds.com/europeanrobins.html"
page = requests.get(URL, timeout=5)
soup = BeautifulSoup(page.content, 'html.parser')  
dirty_text = soup.get_text(" ", strip=False).replace('\n', '.')
sents = text_cleaner(dirty_text)

In [ ]:
html = colorize_prediction(sents[0:5], model=model)

In [ ]:
dirty_text = """
The red cotton tree (Bombax ceiba) is a medium- to large-sized tropical tree that usually grows to an average of 20-25 meters, with old trees up to 60 meters in wet tropical weather. It has a straight tall trunk buttressed at base that bears conical spines to deter animal attacks. Its leaves are deciduous in winter. Crimson red flowers with five petals appear in the spring before the new foliage and are one reason that the tree is widely planted. It produces a capsule which, when ripe, contains white fibres, like cotton, which has been used as a substitute for cotton (hence its common name). 
"""

In [ ]:
sents = text_cleaner(dirty_text)
html = colorize_prediction(sents, model=model, tex=True)
display(HTML(html))

In [ ]:
html = html.replace('\n', '')

In [ ]:
Html_file = open("web_crawler_example","w")
Html_file.write(html)
Html_file.close()

In [ ]:
test = """
Large-billed Flycatcher is most similar to Hill Blue Flycatcher in plumage coloration, but has paler orange throat more contrasting with breast (16). Structural differences include the larger, strongly hook-tipped and broader bill, slightly larger size (11), longer primary projection (10), paler legs, and proportionally shorter tail and legs than Hill Blue Flycatcher (9, 17). Juvenile Large Blue Flycatcher is said to have more colorful upperparts (18). Males of most Chinese Blue Flycatchers and some Blue-throated Flycatcher have distinctly narrower orange-rufous wedge extending up to the dark blue throat and blue across upper throat below jaw-level (7), but the orange notch on throat is often wide and extending up to chin (17).

Male Tickell’s Blue Flycatcher (C. tickelliae) and Indochinese Blue Flycatcher (C. tickelliae) are slightly paler overall, with blue of upperparts not as deep (11) and much less contrasting bright frontal band and carpus edge. Females of both species differ in having bluish-tinged upperparts, albeit often barely perceivable in northern subspecies of Indochinese Blue Flycatcher (C. t. indochina), and markedly blue rump and uppertail, not strongly rufescent as in Hill Blue Flycatcher (19). Note that female Hill Blue Flycatcher occasionally possesses bluish upperparts and rump (19, 7), but it still features more intense rufous orange throat and breast. Indochinese Blue Flycatcher is diagnostic in having a consistently sharp demarcation between orange upper breast and clean white rest of underparts, without any orange on flanks (20, 21, 19).
"""

In [ ]:
text_cleaner(test)